In [56]:
using DataFrames, CSV, JuMP, Gurobi, LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays, Plots, StatsPlots, Distributions

## ____________________________________________________________________

## Import Datasets (Outputted Probabilities and Returns on $100 Bets)

In [57]:
# add actual data here
xgb_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\xgb_cv_pred.csv",DataFrame)
rf_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\rf_pred.csv",DataFrame)
lgbm_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\lgbm_cv_pred.csv",DataFrame);
logreg_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\logreg_pred.csv",DataFrame);
returns = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\data\\group_stage_betting_odds_final.csv",DataFrame);

In [61]:
logreg_preds

Row,draw,home_loss,home_win,home_team,away_team
,Float64,Float64,Float64,String15,String15
1,0.260975,0.00041369,0.738611,Argentina,Saudi Arabia
2,0.618407,0.00605576,0.375537,Argentina,Mexico
3,0.875496,0.0272269,0.0972772,Australia,Denmark
4,0.56212,0.00196917,0.435911,Belgium,Canada
5,0.942321,0.00225783,0.0554213,Belgium,Morocco
6,0.501719,0.0107412,0.48754,Brazil,Serbia
7,0.477244,0.00430241,0.518453,Brazil,Switzerland
8,0.913622,0.0465703,0.0398075,Cameroon,Serbia
9,0.846894,0.137365,0.015741,Cameroon,Brazil


In [53]:
returns_matrix = Matrix(returns[:,3:5]);
# matrix of probabilities for model's predictions
xgb_probs = Matrix(xgb_preds[:,3:5]);
rf_probs = Matrix(rf_preds[:,3:5]);
lgbm_probs = Matrix(lgbm_preds[:,3:5]);
logreg_probs = Matrix(logreg_preds[:,3:5]);

In [54]:
logreg_probs

48×3 Matrix{Any}:
 0.738611   "Argentina"    "Saudi Arabia"
 0.375537   "Argentina"    "Mexico"
 0.0972772  "Australia"    "Denmark"
 0.435911   "Belgium"      "Canada"
 0.0554213  "Belgium"      "Morocco"
 0.48754    "Brazil"       "Serbia"
 0.518453   "Brazil"       "Switzerland"
 0.0398075  "Cameroon"     "Serbia"
 0.015741   "Cameroon"     "Brazil"
 0.0692386  "Canada"       "Morocco"
 ⋮                                   
 0.312992   "Spain"        "Costa Rica"
 0.214121   "Spain"        "Germany"
 0.281705   "Switzerland"  "Cameroon"
 0.310852   "Tunisia"      "Australia"
 0.129742   "Tunisia"      "France"
 0.423506   "USA"          "Wales"
 0.503745   "Uruguay"      "Korea Republic"
 0.0416938  "Wales"        "England"
 0.200552   "Wales"        "Iran"

## ____________________________________________________________________

In [46]:
function FIFA2022_GS_Bettings(P, R, B, K)
    
    # define model
    model = Model(Gurobi.Optimizer)

    # define parameters and decision variables
    m = size(P, 1)              # 48 group stage matches
    n = size(P, 2)              # 3 outcomes

    @variable(model, x[i=1:m, j=1:n] >= 0) # x[i,j] is the amount of money bet on outcome j of match i
    @variable(model, z[i=1:m, j=1:n], Bin) # z[i,j] is whether you bet on outcome j of match i

    # define objective: maximising expected total returns across all matches
    @objective(model, Max, sum(P[i,1]*R[i,1]*x[i,1] + 
                               P[i,2]*R[i,2]*x[i,2] + 
                               P[i,3]*R[i,3]*x[i,3] - 
                               sum(x[i,j] for j=1:n) for i=1:m))

    # # constraints
    # 1) You can only bet on one outcome for each match
    @constraint(model, [i=1:m], sum(z[i,j] for j=1:n) <= 1)

    # 2) x[i,j] <= B*z[i,j] for all i and j
    @constraint(model, [i=1:m, j=1:n], x[i,j] <= B*z[i,j])

    # 3) Budget constraints
    @constraint(model, sum(x[i,j] for j=1:n, i=1:m) <= B)

    # 4) You must spread your risk across at least K matches
    @constraint(model, sum(z[i,j] for j=1:n, i=1:m) >= K)

    # 5) if z = 1, then x > 0
    @constraint(model, [i=1:m, j=1:n], x[i,j] >= 1*z[i,j])

    # solve model
    optimize!(model)

    # get optimal bet amounts (x) and how you bet (z)
    opt_val = objective_value(model)
    x_opt = value.(x)
    z_opt = value.(z)

    return opt_val, x_opt, z_opt
end

FIFA2022_GS_Bettings (generic function with 1 method)

In [47]:
exp_prof, x_opt, z_opt = FIFA2022_GS_Bettings(logreg_probs, returns_matrix, 5000, 5);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-16


MethodError: MethodError: no method matching zero(::Type{String15})
Closest candidates are:
  zero(!Matched::Union{Type{P}, P}) where P<:Dates.Period at C:\Users\shann\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Dates\src\periods.jl:53
  zero(!Matched::SparseArrays.AbstractSparseArray) at C:\Users\shann\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\SparseArrays\src\SparseArrays.jl:42
  zero(!Matched::SA) where SA<:StaticArraysCore.StaticArray at C:\Users\iai\builds\InterpretableAI\SystemImage\SysImgBuilder\.julia\packages\StaticArrays\PUoe1\src\linalg.jl:95
  ...

In [36]:
z_opt

48×3 Matrix{Float64}:
  0.0  -0.0   0.0
 -0.0   0.0   0.0
  0.0   0.0  -0.0
  0.0  -0.0   0.0
  0.0  -0.0   0.0
  0.0  -0.0   0.0
  0.0  -0.0   0.0
  1.0   0.0  -0.0
  0.0   0.0  -0.0
  0.0  -0.0   0.0
  ⋮          
  0.0  -0.0   0.0
 -0.0   0.0   0.0
  0.0  -0.0   0.0
  0.0   0.0  -0.0
  0.0  -0.0   0.0
  0.0  -0.0   0.0
 -0.0   0.0   0.0
  1.0   0.0  -0.0
  0.0  -0.0   0.0

In [38]:
# which matches to bet on
betting_matches = findall(x -> x >= 1, z_opt)

5-element Vector{CartesianIndex{2}}:
 CartesianIndex(8, 1)
 CartesianIndex(11, 1)
 CartesianIndex(28, 1)
 CartesianIndex(29, 1)
 CartesianIndex(47, 1)

In [39]:
exp_prof

52164.207097647

In [43]:
xgb_preds[47,:]

Row,home_team,away_team,home_win,draw,home_loss
,String15,String15,Float64,Float64,Float64
47,Wales,England,0.501899,0.403106,0.0949948
